# Decision Trees

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)


## Construção e modelação

In [2]:
label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])

In [3]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=183)

params =  {'max_iter': [20000], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
grid = GridSearchCV(estimator=clf,
                    param_grid=params,
                    cv=5,
                    n_jobs=1,
                    verbose=2)
grid.fit(X_treino, y_treino)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ...................max_iter=20000, solver=newton-cg; total time=   4.8s
[CV] END ...................max_iter=20000, solver=newton-cg; total time=   4.4s
[CV] END ...................max_iter=20000, solver=newton-cg; total time=   5.2s
[CV] END ...................max_iter=20000, solver=newton-cg; total time=   5.0s
[CV] END ...................max_iter=20000, solver=newton-cg; total time=   7.5s
[CV] END .......................max_iter=20000, solver=lbfgs; total time=  34.8s
[CV] END .......................max_iter=20000, solver=lbfgs; total time=  22.9s
[CV] END .......................max_iter=20000, solver=lbfgs; total time=  24.6s
[CV] END .......................max_iter=20000, solver=lbfgs; total time=  19.3s
[CV] END .......................max_iter=20000, solver=lbfgs; total time=  15.7s
[CV] END ...................max_iter=20000, solver=liblinear; total time=   0.7s
[CV] END ...................max_iter=20000, solve

(0.807463053598747, {'max_iter': 20000, 'solver': 'lbfgs'})

In [5]:
predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/logRegPrediction.csv')